Stopwords source: https://github.etsycorp.com/Engineering/BigData/blob/master/operators/src/main/resources/nlp/stopwords/en-full.txt

In [1]:
!gcloud config set project etsy-bigquery-adhoc-prod


If you have a compatible Python interpreter installed, you can use it by setting
the CLOUDSDK_PYTHON environment variable to point to it.

Updated property [core/project].


In [2]:
%load_ext google.cloud.bigquery

generate table: sr_training_data_queries.sql

In [3]:
%%bigquery df
select distinct data_date, requestUUID, client_query
from `etsy-sr-etl-prod.yzhang.attributed_instance_queries_1229_0104`

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 608425/608425 [00:03<00:00, 158476.87rows/s]


In [4]:
df.head()

,data_date,requestUUID,client_query
0,2023-12-29,00004f09-6fa3-43e0-baee-5ec30e53e604,icons ios
1,2023-12-29,00016f24-7795-47d2-9b41-f75d02d1aabe,disney princess png
2,2023-12-29,0001e116-736a-4dcf-ae02-2eb20c6c9a1f,disney princess thank you tag
3,2023-12-29,00024aa4-1925-41ce-8ef6-703ece70f6d3,niv bible
4,2023-12-29,000252b9-d077-46a5-b457-25a37fab6a37,gift


In [5]:
df.client_query.value_counts()

digital planner                                923
2024 calendar                                  440
engagement gift                                402
stanley name plate                             389
2024 digital planner                           384
                                              ... 
trip png                                         1
taylor signs new home                            1
grandma shirt with grandkid names                1
heirloom quilts                                  1
etsy printable sign pickleball and prosecco      1
Name: client_query, Length: 422696, dtype: int64

# Stopwords

In [19]:
stopwords_full = set()
tmp = []
with open("./stopwords_full.txt", "r") as f:
    for line in f:
        stopwords_full.add(line.strip())
        tmp.append(line.strip())

In [16]:
len(stopwords_full)

612

In [17]:
stopwords_sub = set()
with open("./stopwords.txt", "r") as f:
    for line in f:
        stopwords_sub.add(line.strip())

In [18]:
len(stopwords_sub)

34

In [22]:
def check_stopword_overlap(qstring, stopwd_set):
    query_split = qstring.split(" ")
    for word in query_split:
        if word in stopwd_set:
            return word
    return None

In [23]:
df["stopword_full_overlap"] = df.client_query.apply(check_stopword_overlap, stopwd_set=stopwords_full)
df["stopword_sub_overlap"] = df.client_query.apply(check_stopword_overlap, stopwd_set=stopwords_sub)

In [24]:
df.head()

,data_date,requestUUID,client_query,stopword_full_overlap,stopword_sub_overlap
0,2023-12-29,00004f09-6fa3-43e0-baee-5ec30e53e604,icons ios,None,None
1,2023-12-29,00016f24-7795-47d2-9b41-f75d02d1aabe,disney princess png,None,None
2,2023-12-29,0001e116-736a-4dcf-ae02-2eb20c6c9a1f,disney princess thank you tag,thank,None
3,2023-12-29,00024aa4-1925-41ce-8ef6-703ece70f6d3,niv bible,None,None
4,2023-12-29,000252b9-d077-46a5-b457-25a37fab6a37,gift,None,None


In [29]:
df[~df.stopword_full_overlap.isna()].shape

(118990, 5)

In [31]:
df.data_date.value_counts()

2024-01-02    96922
2024-01-03    95448
2024-01-04    92609
2023-12-29    86395
2023-12-30    85016
2023-12-31    76267
2024-01-01    75768
Name: data_date, dtype: int64

In [30]:
df[~df.stopword_full_overlap.isna()].data_date.value_counts()

2024-01-03    18434
2024-01-02    18114
2024-01-04    17869
2023-12-29    16953
2023-12-30    16781
2023-12-31    16249
2024-01-01    14590
Name: data_date, dtype: int64